In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
from multiprocessing import Process, Pipe
import logging
import time
#import cmath
%matplotlib inline
#import networkx as nx

In [2]:
def first_step_node(c_0,c_1, c_2, g_ii, b_ii, g_i_row, b_i_row, neigh_e_k, neigh_f_k, neigh_lam_k, neigh_mu_k, P_i_d, Q_i_d, P_i_up, P_i_low, Q_i_up, Q_i_low, V_i_up, V_i_low):
    model = ConcreteModel()
    model.IDX = range(4)
    model.x = Var(model.IDX)
    ####%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%######
    #        x = (P_i_g, Q_i_g, e_i, f_i)^T 
    ####%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%######
    #print(model.x[1] *2)
    #print(P_i_d + (model.x[2]**2 + model.x[3]**2)*g_ii + model.x[2] * sum(g_i_row * neigh_e_k - b_i_row * neigh_f_k) + \
    #    model.x[3] * sum(g_i_row * neigh_f_k + b_i_row * neigh_e_k))
    #Nodal equations constraints
    def nodal_eq_P_rule(m):
        return P_i_d + (m.x[2]**2 + m.x[3]**2)*g_ii + m.x[2] * sum(g_i_row * neigh_e_k - b_i_row * neigh_f_k) + \
        m.x[3] * sum(g_i_row * neigh_f_k + b_i_row * neigh_e_k) == m.x[0]
    model.nodal_eq_P = Constraint(rule=nodal_eq_P_rule)
    
    def nodal_eq_Q_rule(m):
        return m.x[1] == Q_i_d - (m.x[2]**2 + m.x[3]**2)*b_ii + m.x[3] * sum(g_i_row * neigh_e_k - b_i_row * neigh_f_k) - \
        m.x[2] * sum(g_i_row * neigh_f_k + b_i_row * neigh_e_k)
    model.nodal_eq_Q = Constraint(rule=nodal_eq_Q_rule)
    
    
    #Limits on variables
    def P_up_lim_rule(m):
        return m.x[0] <= P_i_up
    model.P_up_lim = Constraint(rule=P_up_lim_rule)
    def P_low_lim_rule(m):
        return m.x[0] >= P_i_low
    model.P_low_lim = Constraint(rule=P_low_lim_rule)
    
    def Q_up_lim_rule(m):
        return m.x[1] <= Q_i_up
    model.Q_up_lim = Constraint(rule=Q_up_lim_rule)
    def Q_low_lim_rule(m):
        return m.x[1] >= Q_i_low
    model.Q_low_lim = Constraint(rule=Q_low_lim_rule)
    
    def V_up_lim_rule(m):
        return m.x[2]**2 + m.x[3]**2 <= V_i_up
    model.V_up_lim = Constraint(rule=V_up_lim_rule)
    def V_low_lim_rule(m):
        return m.x[2]**2 + m.x[3]**2 >= V_i_low
    model.V_low_lim = Constraint(rule=V_low_lim_rule)
    
    #Objective
    def value_rule(m):
        dual_vars_len = len(neigh_lam_k)
        return c_0 + c_1 * m.x[0] + c_2 * m.x[0] ** 2 + \
        neigh_lam_k.dot( np.ones(dual_vars_len) * m.x[2] - neigh_e_k) + \
        neigh_mu_k.dot( np.ones(dual_vars_len) * m.x[3] - neigh_f_k) + \
        (rho / 2.) * (sum((np.ones(dual_vars_len) * m.x[3] - neigh_f_k) ** 2) + sum((np.ones(dual_vars_len) * m.x[2] - neigh_e_k)**2))
    
    model.value = Objective(rule=value_rule, sense=minimize)
    
    SolverFactory('ipopt').solve(model, tee=True)
    return np.array([model.x[0].value, model.x[1].value, model.x[2].value, model.x[3].value])

In [15]:
rho = 5.
first_step_node(1., 1., 1., 1., 1.,np.array([1.,1.]), np.array([1.,1.]), np.array([1.,1.]), np.array([1.,1.]), np.array([1.,1.]), np.array([1.,1.]),1.,1.,0.,1.,0.,1.,0.,1.)

Ipopt 3.12.12: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        6
Number of nonzeros in inequality constraint Jacobian.:        8
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

array([ 0.07168896,  0.0583493 ,  0.20775656, -0.25973388])

In [16]:
range(5)[4] * np.ones(5) + range(5)

array([4., 5., 6., 7., 8.])

In [3]:
def second_step_node(g_ii, b_ii, g_i_row, b_i_row, P_i_g_prev, Q_i_g_prev, alpha_i, beta_i, alpha_j, beta_j,  neigh_e_k, neigh_f_k, neigh_lam_k, neigh_mu_k, P_i_d, Q_i_d):
    """
    x - e^i_j vector
    y - f^i_j vector
    """
    model = ConcreteModel()
    l = len(g_i_row)
    model.IDX = range(l)
    model.x = Var(model.IDX)
    model.y = Var(model.IDX)
    ####%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%######
    #        x = (P_i_g, Q_i_g, e_i, f_i)^T 
    ####%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%######
    print(len(model.x))
    #Nodal equations constraints
    def nodal_eq_P_rule(m):
        return P_i_d + (alpha_i**2 + beta_i**2)*g_ii + alpha_i * sum(g_i_row[i] * m.x[i] - b_i_row[i] * m.y[i] for i in range(len(m.x))) + \
        beta_i * sum(g_i_row[i] * m.y[i] + b_i_row[i] * m.x[i] for i in range(len(g_i_row))) == P_i_g_prev
    model.nodal_eq_P = Constraint(rule=nodal_eq_P_rule)
    
    def nodal_eq_Q_rule(m):
        return Q_i_g_prev == Q_i_d - (alpha_i**2 + beta_i**2)*b_ii + beta_i * sum(g_i_row[i] * m.x[i] - b_i_row[i] * m.y[i] for i in range(len(b_i_row))) - \
        alpha_i * sum(g_i_row[i] * m.x[i] + b_i_row[i] * m.y[i] for i in range(len(g_i_row)))
    model.nodal_eq_Q = Constraint(rule=nodal_eq_Q_rule)
    
    
    
    
    #Objective
    def value_rule(m):
        dual_vars_len = len(neigh_lam_k)
        return neigh_lam_k.dot(alpha_j - m.x) + neigh_mu_k.dot(beta_j - m.y) + \
        (rho / 2.) * (sum(alpha_j - m.x)**2 + sum(beta_j - m.y)**2)
    
    model.value = Objective(rule=value_rule, sense=minimize)
    
    SolverFactory('ipopt').solve(model, tee=True)
    res_es = []
    for i in range(len(model.x)):
        res_es.append(model.x[i].value)
    res_fs = []
    for i in range(len(model.y)):
        res_fs.append(model.y[i].value)
    return res_es, res_fs

In [18]:
rho = 5.
second_step_node(1., 1., np.array([1.,1.]), np.array([1.,1.]), 0.5, 0.5, 0.6, 0.9, np.array([1.,1.]), np.array([1.,1.]), np.array([1.,1.]), np.array([1.,1.]), np.array([1.,1.]), np.array([1.,1.]), 1., 1.)


2
Ipopt 3.12.12: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.12, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        8
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
T

([-0.49230769230769234, -0.49230769230769234],
 [-0.3217948717948718, -0.3217948717948717])

In [4]:
def third_step_node(neigh_lam_k, neigh_mu_k, neigh_e_j_rec, neigh_f_j_rec, neigh_e_j, neigh_f_j):
    neigh_lam_k_next = neigh_lam_k + rho * (neigh_e_j_rec - neigh_e_j)
    neigh_mu_k_next = neigh_mu_k + rho * (neigh_f_j_rec - neigh_f_j)
    return (neigh_lam_k_next, neigh_mu_k_next)

In [20]:
(1 + 1j).imag

1.0

In [21]:
test_Y = np.array([
    [1 + 1j, 1 - 0.8j],
    [1 - 0.8j, 1 + 0.5j]
])

In [22]:
test_G = test_Y.real
test_B = test_Y.imag

In [23]:
test_costs = np.array([
    [1., 2.],
    [3.,4.],
    [5.,6.]
])
test_costs[1,:]

array([3., 4.])

In [24]:
len(test_Y)

2

In [25]:
def get_non_zero_el_subvec(a):
    tmp = []
    for el in a:
        if el != 0:
            tmp.append(el)
    return np.array(tmp)

test_a = [1., 0., 1e-10, 2]
print(get_non_zero_el_subvec(test_a))

[1.e+00 1.e-10 2.e+00]


In [5]:
def get_neigh_numbers(i, Y):
    res = []
    for j in range(len(Y)):
        if (Y[i,j] and i != j) != 0:
            res.append(j)
    return res
def get_non_zero_el_nums(a):
    res =[]
    for i in range(len(a)):
        if a[i] != 0:
            res.append(i)
    return res

In [39]:
test_listl = []
test_listl.append([1,2])
test_listl.append([1,2])
test_listl[1][1]

2

In [7]:
def get_inc_matrix(Y):
    pipe_conns = np.zeros((len(Y), len(Y)))
    for i in range(len(Y)):
        for j in range(len(Y)):
            if (j >= i):
                pipe_conns[i,j] = Y[i,j]
    print(pipe_conns)
    Inc = []
    for i in range(len(Y)):
        for j in range(len(Y)):
            if ((i != j) and (pipe_conns[i,j] != 0)):
                tmp = np.zeros(len(Y))
                tmp[i] = -1
                tmp[j] = 1
                Inc.append(tmp)
                
        
    return np.array(Inc)

In [82]:

testY = np.array([
    [1, 2, 1],
    [2, 1, 3],
    [1, 3, 1]
])
print(get_inc_matrix(testY))

[[1. 2. 1.]
 [0. 1. 3.]
 [0. 0. 1.]]
[[-1.  1.  0.]
 [-1.  0.  1.]
 [ 0. -1.  1.]]


In [84]:
def get_pipes(Y):
    pipe_par = []
    pipe_child = []
    inc = get_inc_matrix(Y)
    for i in range(len(Y)):
        pipe_par.append([])
        pipe_child.append([])
    
    for i in range(inc.shape[0]):
        nums = get_non_zero_el_nums(inc[i])
        parent, child = Pipe()
        pipe_par[nums[0]].append(parent)
        pipe_child[nums[1]].append(child)
            
    
    return (pipe_par, pipe_child)

In [28]:
def fully_decentr_opf(costs, G, B, P_d, Q_d, P_up, P_low, Q_up, Q_low, V_up, V_low, iter_num=300)
    logging.basicConfig(level=logging.DEBUG,
                        format='%(asctime)s (%(threadName)-2s) %(message)s',
                        )
    def node_first(c_0_i, c_1_i, c_2_i, g_ii, b_ii, g_i_row, b_i_row, neigh_e_k, neigh_f_k, neigh_lam_k, neigh_mu_k, P_i_d, Q_i_d, P_i_up, P_i_low, Q_i_up, Q_i_low, V_i_up, V_i_low):
        res = first_step_node(c_0_i, c_1_i, c_2_i, g_ii, b_ii, g_i_row, b_i_row, neigh_e_k, neigh_f_k, neigh_lam_k, neigh_mu_k, P_i_d, Q_i_d, P_i_up, P_i_low, Q_i_up, Q_i_low, V_i_up, V_i_low)
        return res
    def node_second(g_ii, b_ii, g_i_row, b_i_row, P_i_g_prev, Q_i_g_prev, alpha_i, beta_i, alpha_j, beta_j,  neigh_e_k, neigh_f_k, neigh_lam_k, neigh_mu_k, P_i_d, Q_i_d):
        res = second_step_node(g_ii, b_ii, g_i_row, b_i_row, P_i_g_prev, Q_i_g_prev, alpha_i, beta_i, alpha_j, beta_j,  neigh_e_k, neigh_f_k, neigh_lam_k, neigh_mu_k, P_i_d, Q_i_d)
        return res
    def node_third(neigh_lam_k, neigh_mu_k, neigh_e_j_rec, neigh_f_j_rec, neigh_e_j, neigh_f_j):
        res = third_step_node(neigh_lam_k, neigh_mu_k, neigh_e_j_rec, neigh_f_j_rec, neigh_e_j, neigh_f_j)
        return res
    
    
    def run(par_pipes, child_pipes, c_0_i, c_1_i, c_2_i, g_ii, b_ii, g_i_row, b_i_row, neigh_e_k, neigh_f_k, neigh_lam_k, neigh_mu_k, P_i_d, Q_i_d, P_i_up, P_i_low, Q_i_up, Q_i_low, V_i_up, V_i_low):
        # barrier here
        for j in range(iter_num):
            res1 = first_step_node(c_0_i, c_1_i, c_2_i, g_ii, b_ii, g_i_row, b_i_row, neigh_e_k, neigh_f_k, neigh_lam_k, neigh_mu_k, P_i_d, Q_i_d, P_i_up, P_i_low, Q_i_up, Q_i_low, V_i_up, V_i_low)
            #barrier here
            
        
    if __name__ == '__main__':
        num_proc = len(Y)
        parent_pipes, child_pipes = get_pipes(Y)
        
        for i in range(num_proc):
            c_0_i = costs[i,0]
            c_1_i = costs[i,1]
            c_2_i = costs[i,2]
            g_ii  = G[i,i]
            b_ii  = B[i,i]

            g_i_row = get_non_zero_el_subvec(G[i,:])
            b_i_row = get_non_zero_el_subvec(B[i,:])

            P_i_d   = P_d[i]
            Q_i_d   = Q_d[i]

            P_i_up    = P_up[i]
            P_i_low   = P_low[i]

            Q_i_up    = Q_up[i]
            Q_i_low   = Q_low[i]

            V_i_up    = V_up[i]
            V_i_low   = V_low[i]
            

            # initial guesses
            P_i_g = 1.
            Q_i_g = 1.
            e_i   = 1.
            f_i   = 1.
            neigh_e_k = np.ones(len(g_i_row))
            neigh_f_k = np.ones(len(g_i_row))
            neigh_lam_k = np.ones(len(g_i_row))
            neigh_mu_k = np.ones(len(g_i_row))
            
            
            p = Process(target=run, args=(parent_pipes[i], child_pipes[i],par_pipes, child_pipes, c_0_i, c_1_i, c_2_i, g_ii, b_ii, g_i_row, b_i_row, neigh_e_k, neigh_f_k, neigh_lam_k, neigh_mu_k, P_i_d, Q_i_d, P_i_up, P_i_low, Q_i_up, Q_i_low, V_i_up, V_i_low,))
            p.start()    
            
            


SyntaxError: invalid syntax (<ipython-input-28-f1596afc43f4>, line 1)

In [23]:
list(np.ones(3))

[1.0, 1.0, 1.0]

In [33]:
from multiprocessing import Process, Pipe

def f(conn):
    conn.send([42, None, 'hello'])
    for i in range(100):
        pass
    conn.send([425, None, 'hello'])
    conn.close()
    
if __name__ == '__main__':
    parent_conn, child_conn = Pipe()
    p = Process(target=f, args=(child_conn,))
    p.start()
    print(parent_conn.recv())   # prints "[42, None, 'hello']"
    print(parent_conn.recv()) 
    p.join()

[42, None, 'hello']
[425, None, 'hello']
